In [1]:
# This is a demo of using vessel_analysis_3d to perform quantitative analysis
# on the data released with VesselExpress: https://zenodo.org/record/6025935#.ZCUzu-zP0SI

# suppose the data has been downloaded from Zenodo and saved at base_path
from pathlib import Path
import numpy as np
import pandas as pd
from vessel_analysis_3d.processing_pipeline import Pipeline3D
from skimage.morphology import skeletonize
from aicsimageio import AICSImage

base_path = Path("/mnt/eternus/project_data/vesselexpress")
data_path = base_path / Path("Figure3")

In [2]:
PARAMS = {
    "pixelDimensions": [2.0, 1.015625, 1.015625],
    "pruningScale": 1.5,
    "lengthLimit": 3,
    "diaScale": 2,
    "branchingThreshold": 0.25
}

In [3]:
area_list = ["Cortex", "Hippocampus", "Midbrain", "Striatum"]
all_stats = []
fil_stats = []
brp_stats = []
for area in area_list:
    set_path = data_path / f"{area}" / Path("Binary")
    filenames = sorted(set_path.glob("Binary*.tiff"))
    for fn in filenames:
        SEG = AICSImage(fn).get_image_data("ZYX", C=0, T=0)

        fn_base = fn.stem[7:]
        if fn_base.startswith("control"):
            group_tag = "control"
        elif fn_base.startswith("ob") or fn_base.startswith("OB"):
            group_tag = "obesity"
        else:
            raise ValueError(f"unrecognized group name {fn_base}")

        # get preliminary skeleton
        SKL = skeletonize(SEG > 0, method="lee")
        SKL = SKL.astype(np.uint8)
        SKL[SKL > 0] = 1

        # get stats
        _, _, _, reports = Pipeline3D.process_one_file(SEG, SKL, param_dict=PARAMS, basename=fn_base)

        tab_all = reports[0]
        tab_all["area"] = area
        tab_all["group"] = group_tag
        all_stats.append(tab_all)

        tab_fil = reports[1]
        tab_fil["area"] = area
        tab_fil["group"] = group_tag
        fil_stats.append(tab_fil)

        tab_brp = reports[2]
        tab_brp["area"] = area
        tab_brp["group"] = group_tag
        brp_stats.append(tab_brp)

df_all = pd.concat(all_stats)
df_fil = pd.concat(fil_stats)
df_brp = pd.concat(brp_stats)

/mnt/eternus/users/Jianxu/projects/vessel_analysis_3d/vessel_analysis_3d/graph/filament.py:345: RuntimeWarning: invalid value encountered in double_scalars
  curveDisplacement / segLength


In [5]:
df_all.to_csv("basic_stats.csv", index=False)
df_fil.to_csv("filament_stats.csv", index=False)
df_brp.to_csv("branching_point_stats.csv", index=False)